In [1]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import matplotlib.dates as mdates
from matplotlib.collections import LineCollection
from matplotlib.colors import Normalize

In [2]:
hydro_number = 2
depth_choice = 1300  # 50, 100, 500, 1300
out_path = f"/home/imonge/Documents/Abiotic/current_speed_different_depths_h{hydro_number}_depth{depth_choice}.png"
hydro_data = xr.open_dataset(f"../../../../../data/test_correlation/ERA5_concat/ERA5_MAHY*{hydro_number}_current_merged.nc") # Path of a directory containing the hydrology data (copernicus) for the station selected

date_start_current = datetime.datetime(2020,10,16,0,11,45)# exemple : début le 10/10/21 à 5h30 et 25s
date_end_current = datetime.datetime(2024,9,26,12,46,24)

In [6]:
## Plot
plt.figure(figsize=(18,10))
time_data = hydro_data["time"].values
time_data = np.array([np.datetime64(t).astype('datetime64[s]').astype(datetime.datetime) for t in time_data])
mask = (time_data >= date_start_current) & (time_data <= date_end_current)
#time_data = np.array(time_data)[mask]

depth_index = np.argmin(abs(hydro_data["depth"].data-depth_choice)) # Finds the index in the dataset corresponding to the depth chosen
uo = np.array(hydro_data["uo"][:,depth_index])[mask].flatten()
vo = np.array(hydro_data["vo"][:,depth_index])[mask].flatten()

# Speed and direction
speed = np.sqrt(uo**2+vo**2)
angle_rad = np.arctan2(uo, vo) # uo = north vo = east
angle_deg= np.degrees((angle_rad) + 360) % 360 # 0-360°, 0° towards north

x = mdates.date2num(time_data[mask])
y = speed

points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

norm = Normalize(vmin=0, vmax=360)
lc = LineCollection(segments, cmap='hsv', norm=norm)
lc.set_array(angle_deg[:-1])
lc.set_linewidth(2)

fig, ax = plt.subplots(figsize=(18, 8))
ax.add_collection(lc)
ax.set_xlim(x.min(), x.max())
ax.set_ylim(0, np.nanmax(speed) * 1.1)

ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))


plt.title(f"Current speed (depth {depth_choice} m) - color = direction [°]")
plt.xlabel("Date")
plt.ylabel("Current speed (m/s)")

# Add colorbar
cbar = plt.colorbar(lc, ax=ax)
cbar.set_label('Current direction (° from North)')

plt.tight_layout()
plt.grid(True)

# Show plot
%matplotlib qt
plt.show()

## Save plot
# plt.savefig(out_path)